# SqlAlchemy and Pandas

The following is a personal list of small things to speed up some data table creation/manipulation. 

Not everything is yet on the cloud and SqlAlchemy still provides some powerful tools to work with Relational Databases. This is perhaps something to help the new data scientist dealing with CRUD tasks while using pandas dataframes.

<ol>
    <li>Citations</li>
        <ul>
            <li>Python ...</li>
            <li>Pandas ...</li>
            <li>SqlAlchemy ...</li>
        </ul>
    <li>Resources</li>
    <ul>
        <li>
            <a href="https://www.pythonsheets.com/notes/python-sqlalchemy.html">SqlAlchemy Cheatsheet</a>
        </li>
    </ul>
</ol>

In [1]:
import json
import pandas as pd
import sqlalchemy as sa

from collections import namedtuple
from sqlalchemy.ext.declarative import declarative_base

In [2]:
Base = declarative_base()
engine = sa.create_engine('sqlite:///')

In [3]:
class Song(Base):
    __tablename__ = 'songs'
    
    name = sa.Column(sa.String(100), primary_key=True)
    artist = sa.Column(sa.String(100), primary_key=True)
    length = sa.Column(sa.Float, nullable=False)                  # the length in seconds
    language = sa.Column(sa.String(20), nullable=False)         
    country_of_release = sa.Column(sa.String(20), nullable=True)  # the default of nullable is True

Let me start by saying that **Primary Keys** are very important in a relational database. Some convenient methods like `pandas.DataFrame.to_sql` may not always workout when duplicates are mistakenly inserted.

Bare in mind that most people will stay away from any property that begins with `"__"` but in the case of the class above, the `__table__` method could be accessed by reflecting an existing table so I will make use of it. It is important to know the dangers associated with the `sa.Table` object; for example it has the `drop` property which can do serious damage if used accidentally.

As you already know, you can create the table if it does not exist and you can insert data like so:

In [4]:
# create
Song.__table__.create(engine, checkfirst=True)

In [5]:
# insert
session = sa.orm.Session(engine)
song_1 = Song(name="Bohemian Rhapsody", artist="Queen", length=355, language="English")
session.add(song_1) # there is an add_all method that adds a list of many instances
session.commit()
session.close()

In [6]:
pd.read_sql('songs', engine)

,name,artist,length,language,country_of_release
0,Bohemian Rhapsody,Queen,355.0,English,None


## Inserting directly from Pandas DataFrames

Everything above is very trivial and can easily be done with a pandas DataFrame

In [7]:
songs = pd.DataFrame({"name": ["song 1", "song 2"],
                      "artist": ["artist 1", "artist 1"],
                      "length": [300, 240],
                      "language": ["English", "French"],
                      "country_of_release": [None, "Canada"]})
songs

,name,artist,length,language,country_of_release
0,song 1,artist 1,300,English,None
1,song 2,artist 1,240,French,Canada


### Instances from the dataframe

In [8]:
records = songs.to_dict(orient="records")
records

[{'name': 'song 1',
  'artist': 'artist 1',
  'length': 300,
  'language': 'English',
  'country_of_release': None},
 {'name': 'song 2',
  'artist': 'artist 1',
  'length': 240,
  'language': 'French',
  'country_of_release': 'Canada'}]

In [9]:
song_instances = [Song(**record) for record in records]
song_instances[0]

In [10]:
session = sa.orm.Session(engine)
session.add_all(song_instances)
session.commit()
session.close()

In [11]:
pd.read_sql('songs', engine)

,name,artist,length,language,country_of_release
0,Bohemian Rhapsody,Queen,355.0,English,None
1,song 1,artist 1,300.0,English,None
2,song 2,artist 1,240.0,French,Canada


### Using `__table__`

In [12]:
print(Song.__table__.insert())

INSERT INTO songs (name, artist, length, language, country_of_release) VALUES (:name, :artist, :length, :language, :country_of_release)


In [13]:
songs = pd.DataFrame({"name": ["song 3", "song 4"],
                      "artist": ["artist 1", "artist 2"],
                      "length": [300, 240],
                      "language": ["English", "French"],
                      "country_of_release": [None, "Canada"]})
songs

,name,artist,length,language,country_of_release
0,song 3,artist 1,300,English,None
1,song 4,artist 2,240,French,Canada


In [14]:
inserted = engine.execute(Song.__table__.insert(), songs.to_dict(orient="records"))
print(f"Inserted {inserted.rowcount} in {Song.__tablename__}")

Inserted 2 in songs


In [15]:
pd.read_sql('songs', engine)

,name,artist,length,language,country_of_release
0,Bohemian Rhapsody,Queen,355.0,English,None
1,song 1,artist 1,300.0,English,None
2,song 2,artist 1,240.0,French,Canada
3,song 3,artist 1,300.0,English,None
4,song 4,artist 2,240.0,French,Canada


One thing to keep in mind is the `__table__` approach above does not work well with `numpy.nan`, `numpy.NaN` or `pd.NaT`. It's best to use the `pandas.DataFrame.where(pandas.notnull(...), None)` to replace all missing values with the python default for null, `None`

Alright great. Nothing interesting so far. But it is very possible to have some bad data that go right through the craks. Since we defined a class to represent our table, we can do many more things with it. 

For starters, we can create an `__init__` method so that we do not have to provide `**kwargs` to create instances.

We can also spice things up by adding methods to create some quick `json` docs of the table. We can also create some simple validators for our data

In [16]:
class SongImproved(Base):
    __tablename__ = 'songs_improved'
    
    name = sa.Column(sa.String(100), primary_key=True)
    artist = sa.Column(sa.String(100), primary_key=True)
    length = sa.Column(sa.Float, nullable=False)                  # the length in seconds
    language = sa.Column(sa.String(20), nullable=False)         
    country_of_release = sa.Column(sa.String(20), nullable=True)  # the default of nullable is True
    
    def __init__(self, name: str, artist: str, length: float, language: str, country_of_release: str = None):
        self.name = name
        self.artist = artist
        self.length = length
        self.language = language
        self.country_of_release = country_of_release
        
    @property
    def primary_constraint(self):
        """
        """
        return f"name: {self.name}, artist: {self.artist}"
    
    @classmethod
    def column_order(cls):
        """
        This is provides the columns in the order defined since we used the same order in the
        __init__ method. This is useful when your dataframe has the same columns but in different order
        """
        return [str(column.name) for column in cls.__table__.columns]
        
    @classmethod
    def my_simple_docs(cls):
        """
        Create a simple json doc of the table
        """
        my_dict = {
            "table_name": cls.__tablename__,
            "columns": [
                {
                    "name": str(column.name),
                    "type": str(column.type),
                    "primary_key": column.primary_key,
                    "nullable": column.nullable,                    
                }
                for column in cls.__table__.columns
            ]
        }
        
        return json.dumps(my_dict)
    
    @property
    def my_simple_validator(self):
        """
        This property can only be used before data instance is inserted or added to a session
        """
        faults = []
        Validated = namedtuple('Validated', ['validated', 'faults'])
        attributes = (str(column.name) for column in self.__table__.columns)
        
        for attribute in attributes:
            # if the property should be a string, we need to check the length as well
            value, column = getattr(self, attribute), self.__table__.columns[attribute]
            
            if pd.isnull(value) and column.nullable:
                continue
                
            if isinstance(value, column.type.python_type):
                if isinstance(value, str) and len(value) > column.type.length:
                    faults.append(
                        f"Song: {self.primary_constraint}: value {value} has length {len(value)} \
                        but {column.type.length} is the max"
                    )
            else:
                faults.append(
                    f"Song: {self.primary_constraint}: {value} is of type {type(value)} but \
                    {column.type.python_type} was expected"
                )
                
        return Validated(not faults, faults)

In [17]:
# Documentation - very simple but useful
SongImproved.my_simple_docs()

'{"table_name": "songs_improved", "columns": [{"name": "name", "type": "VARCHAR(100)", "primary_key": true, "nullable": false}, {"name": "artist", "type": "VARCHAR(100)", "primary_key": true, "nullable": false}, {"name": "length", "type": "FLOAT", "primary_key": false, "nullable": false}, {"name": "language", "type": "VARCHAR(20)", "primary_key": false, "nullable": false}, {"name": "country_of_release", "type": "VARCHAR(20)", "primary_key": false, "nullable": true}]}'

In [18]:
songs = pd.DataFrame({"name": ["song 3", "song 4"],
                      "artist": ["artist 1", "artist 2"],
                      "length": [300.0, "240"],
                      "language": ["English", "French         .             ,"],
                      "country_of_release": [None, "Canada"]})
songs

,name,artist,length,language,country_of_release
0,song 3,artist 1,300,English,None
1,song 4,artist 2,240,"French . ,",Canada


In [19]:
song_instances = [SongImproved(*values) for values in songs[SongImproved.column_order()].values]

# we can check which instances are valided directly like this
[song_instance.my_simple_validator.validated for song_instance in song_instances]

[True, False]

In [20]:
# as expected, the first row is totally fine
song_instances[0].my_simple_validator

Validated(validated=True, faults=[])

In [21]:
# The second row is not correct and we can see two issues with it
song_instances[1].my_simple_validator

Validated(validated=False, faults=["Song: name: song 4, artist: artist 2: 240 is of type <class 'str'> but                     <class 'float'> was expected", 'Song: name: song 4, artist: artist 2: value French         .             , has length 30                         but 20 is the max'])

With some additonal work upfront, it is easy to catch instances where something went wrong. These examples are simple enough that they would have been noticed during data processing but eventually something slips through the cracks and it's nice to have some additional line of defense.

## Updating data using Pandas DataFrames

I've noticed a couple of friends struggling with updating tables directly with dataframes. It is actually very easy to use SqlAlchemy Table objects to update records. The ORM objects can be used as well but I'll just skip over that

In [22]:
...

Ellipsis

## Using Sqlalchemy over Raw SQL

Sqlalchemy is not a replacement for raw sql strings. There are situations where you could create your complex sql queries directly with SqlAlchemy objects. One of the benefits of that is that you can actually refractor your object (if someone changed a column name) and all the fields will be updated as needed. As opposed to the sql query string where you may have to use the `ctrl+F` shortcut.

What I like most about using the SqlAlchemy objects is that I can combine many of them using `f-strings`. This is manily because you could obtain the query strings created with SqlAlchemy so that your teammates do not have to learn SqlAlchemy; they can just read the sql query strings.

In [23]:
...

Ellipsis